In [1]:
import numpy as np
import random
import pandas as pd
import math


In [2]:
for i in range(125):
    df = pd.read_csv(f'./finaldatasetgz/googleclusterdata{i}.csv.gz')
    
    if i==0:
        data= df.copy()

    else :
        data= pd.concat([data,df],axis=0,ignore_index=True)

    print(i)    

print(df)     

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
        Unnamed: 0     timestamp sizecpu sizeram  timeslot  gs   ds  cloud
0         47987876  2.479339e+12       s       l     688.0   1  150      5
1         47987877  2.479339e+12       s       l     688.0   1  300      3
2         47987878  2.479339e+12       s       l     688.0   1  150      0
3         47987879  2.479339e+12       s       l     688.0   1  300      3
4         47987880  2.479339e+12       s       l     688.0   2  150      8
...            ...           ...     ...     ...       ...  ..  ...    ...
386994    48374870  2.506197e+12       m       s     696.0   2  150      7
386995    

In [3]:
data

,Unnamed: 0,timestamp,sizecpu,sizeram,timeslot,gs,ds,cloud
0,0,0.000000e+00,l,l,0.0,1,150,4
1,1,0.000000e+00,l,l,0.0,2,300,8
2,2,0.000000e+00,m,l,0.0,1,150,9
3,3,0.000000e+00,m,l,0.0,2,300,0
4,4,0.000000e+00,m,l,0.0,2,300,5
...,...,...,...,...,...,...,...,...
48374870,48374870,2.506197e+12,m,s,696.0,2,150,7
48374871,48374871,2.506197e+12,m,s,696.0,2,150,0
48374872,48374872,2.506198e+12,m,s,696.0,1,150,3
48374873,48374873,2.506200e+12,m,s,696.0,1,150,6


In [4]:
class jobreq:
    #S types of Jobs serviced at each cloud, 
    # each specified by a three-tuple < ms,gs,ds>    
    def __init__(self,cloud,ms,gs,ds,jobtype):
        #request for cloud
        self.cloud = cloud
        #type of required VM
        self.ms=ms
        # # of VMs required+
        self.gs=gs
        # SLA in terms of Response time
        self.ds=ds
        #job type S
        self.jobtype = jobtype    

In [5]:
# def randomsamplingHTD(length,clouds,requireddata):
#     a=2.0
#     dist = np.random.power(a, size = length)
#     scaled_dist = np.round(dist*9).astype(int)
#     values = np.random.choice(scaled_dist,size=length)
#     sample = pd.DataFrame(values,columns=['cloud'],index = list(requireddata.index.values))
#     sample = pd.concat([requireddata,sample],axis=1)
    
#     return sample
    

In [6]:
def jobgenerator(df,timeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM):
    requests = []
#     requireddata =df.loc[df['timeslot'] == timeslot+1]
    condition = df["timeslot"] == timeslot+1
    requireddata = df[condition].copy()
#     requireddata['cloud'] = np.random.randint(10,size = requireddata.shape[0])
    
    for i in range(len(requireddata)):
        m1=requireddata['sizeram'].iloc[i]
        m2=requireddata['sizecpu'].iloc[i]
        m = f'{m1}{m2}'
        g = requireddata['gs'].iloc[i]
        d = requireddata['ds'].iloc[i]
        c = requireddata['cloud'].iloc[i]
        t = (4*possibleVMtypes.index(m))+(2*possiblenoofVM.index(g))+possibleDS.index(d)
        requests.append(jobreq(c,m,g,d,t))
    return requests

In [7]:
class CSPs:
  
    def __init__(self,name,nMI,betaIT,cuspay,typeofServiceS=24):
        self.nMI=nMI
        self.name=name
        #buy-bid to be submitted to auctioneer    
        self.buybid={}
        #sell-bid to be submitted to auctioneer
        self.sellbid={}
         #queue maintained by cloud
        self.typeofServiceS=typeofServiceS
        self.quelen = [0 for i in range(typeofServiceS)]
        self.possibleVMtype = ['ss','sm','sl','ls','lm','ll']
        self.possibleDS = [150,300]
        self.possiblenoofVM = [1,2]
        self.V=9000000
        #virtual queue maintained by cloud
        self.vQue = [0 for i in range(typeofServiceS)]
        # # of jobs dropped
        self.D = [0 for i in range(typeofServiceS)]
        #cost of running the server
        self.betaIT=betaIT
#         self.cusPay = [0.02,0.02,0.02,0.04,0.04,0.04]
        self.cusPay = cuspay
        self.cMI = [30,30,30,10,10,10]
        self.allService = self.obtainServiceSdesc()
        self.maxNoofJobdrop=1000
        self.xi=cuspay
        self.pays = [0 for i in range(5)]
        self.totalpay = 0
        
        
    def obtainServiceSdesc(self):
        possibleServiceS ={}
        x=0
        for i in possibleVMtypes:
            for j in possiblenoofVM:
                for k in possibleDS:
                    possibleServiceS[x]=[i,j,k]
                    x=x+1     
        return possibleServiceS


    def initqueue(self,rsit):
        for i in rsit[self.name]:
            self.quelen[i.jobtype] = self.quelen[i.jobtype]+1
            self.updatepayoff(0,(i.gs*self.cusPay[self.possibleVMtype.index(i.ms)]))
                    
    def keyAddition(self,dicts,key,value):
        if key not in list(dicts.keys()):
            temp=[]
            temp.append(value)
            dicts[key]=temp
        else:
            dicts[key].append(value)
        return dicts
    
    
    def getBids(self):
        self.wsdashi={}
        for i in range(len(self.quelen)):        
            if self.quelen[i]==0:
                self.wsdashi = self.keyAddition(self.wsdashi,self.allService[i][0],0)
                continue        
            else:
                self.wsdashi = self.keyAddition(self.wsdashi,self.allService[i][0],(self.quelen[i]+self.vQue[i])/self.allService[i][1])
        
        self.sStarM={}      
        for key,value in self.wsdashi.items():
            index = (self.typeofServiceS/len(self.possibleVMtype))*self.possibleVMtype.index(key)+value.index(max(value))
            self.sStarM[key]=index
            self.buybid[key]=max(value)/self.V
            self.sellbid[key]=max(max(value)/self.V,self.betaIT/self.cMI[self.possibleVMtype.index(key)])
#         print("Buybid:",self.buybid) 
#         print("Sellbid:",self.sellbid)
#         print("sStar:",self.sStarM)
#         print("wsdashi:",self.wsdashi)

    
    def notosell(self,ind):
        return self.cMI[ind]*self.nMI[ind]
      
    def queueupdate(self,muSIJ,rsit):
        for services in range(len(muSIJ)):
            summation = sum(muSIJ[services][self.name])
            if summation >= self.quelen[services]:
                self.quelen[services]=0
            else:
                self.quelen[services] = self.quelen[services] - summation 
        self.initqueue(rsit)
        
    def vqueueupdate(self,muSIJ,csps):
        for services in range(len(muSIJ)):
            msInd=self.possibleVMtype.index(self.allService[services][0])
            ds = self.allService[services][2]
            gs=self.allService[services][1]
            
            if ds==150:
                epsilonS = 1000
            elif ds==300:
                epsilonS=500
            
            if self.quelen[services]==0:
                t=1
            else:
                t=0
            temp = self.vQue[services] + (1-t)*(epsilonS-sum(muSIJ[services][self.name])) - t*sum([(i.cMI[msInd]*i.nMI[msInd])/gs for i in csps])
            self.vQue[services]=max(0,temp)
            
            
    def dropjobs(self):
        for services in range(self.typeofServiceS):
            tempxi =self.xi[self.possibleVMtype.index(self.allService[services][0])]*self.allService[services][1]
            if (self.quelen[services]+self.vQue[services]) > (self.V*tempxi):
                self.D[services]=self.D[services]+self.maxNoofJobdrop
                self.quelen[services]=max(0,self.quelen[services]-self.maxNoofJobdrop)
                self.vQue[services]=max(0,self.vQue[services]-self.maxNoofJobdrop)
                self.updatepayoff(3,tempxi*1000)
                            
    def updatepayoff(self,ind,pay):
        self.pays[ind]= self.pays[ind]+pay
        
    def provisioningcost(self,nmi):
        for keys,values in nmi.items():
            self.pays[2]= self.pays[2]+ self.betaIT*values[self.name]
            
    def totalpayment(self):
        self.totalpay = self.pays[0]+self.pays[1]-self.pays[2]-self.pays[3]-self.pays[4]
            
    def dropinend(self):
        for services in range(self.typeofServiceS):
#             print(self.quelen[services]*self.cusPay[self.possibleVMtype.index(self.allService[services][0])])
            self.pays[0]= self.pays[0]- (self.quelen[services]*self.cusPay[self.possibleVMtype.index(self.allService[services][0])]*self.allService[services][1])
    

        
        
        

In [8]:
class auctioneer:
         
    Buybid={}
    Sellbid={}
    possibleVMtype = ['ss','sm','sl','ls','lm','ll']
    
    def __init__(self,CSP):
        self.csps = CSP
        
    def allbidsandasks(self):
        for i in self.possibleVMtype:
            self.Buybid[i]={}
            self.Sellbid[i]={}
            for c in self.csps:
                self.Buybid[i][c.name]=c.buybid[i]
                self.Sellbid[i][c.name]=c.sellbid[i]
    
    def printbids(self):
        print("Buybids:",self.Buybid)
        print("SellBids:",self.Sellbid)
        
    def sortBid(self):
        for key in self.Buybid:
            self.Buybid[key]=dict(sorted(self.Buybid[key].items(),reverse=True, key=lambda item: item[1]))
            
        for key in self.Sellbid:
            self.Sellbid[key]=dict(sorted(self.Sellbid[key].items(),key=lambda item: item[1]))
            
            
    def winnerDetermination(self):
        self.winningBuyer={}
        self.winningSeller={}
        for key,value in self.Buybid.items():
            self.winningBuyer[key]={}
            s=sorted(set(list(value.values())))
            self.winningBuyer[key][list(value.items())[0][0]]=s[-2]
        print("WinningBuyer:",self.winningBuyer)
        
        for key,value in self.Sellbid.items():
            setofwinningseller=[]
            clearingprice = list(self.winningBuyer[key].values())[0]
            clearingsellprice=0

            for keys,values in value.items():
                if values <= clearingprice:
                    setofwinningseller.append(keys)
                    clearingsellprice=values
                    
            if len(setofwinningseller) != 0:   
                setofwinningseller.pop()
            self.winningSeller[key]=[]
            self.winningSeller[key].append(setofwinningseller)
            self.winningSeller[key].append(clearingsellprice)            
        print("WinningSeller:",self.winningSeller)
            
        
        # of VMs sold from cloud j to cloud i
        
    def determineAlpha(self):
        self.alphaMIJ = {}
        for size in self.possibleVMtype:
            self.alphaMIJ[size]=[[0 for i in range(len(self.csps))] for j in range(len(self.csps))]
            temp =[0 for x in range(len(self.csps))]
            for i in range(len(self.csps)):
                if i in self.winningSeller[size][0]:
                    temp[i]=self.csps[i].notosell(self.possibleVMtype.index(size))
            self.alphaMIJ[size][list(self.winningBuyer[size].keys())[0]]= temp.copy()
#         print("# of VMs sold j->i alphaMIJ:",self.alphaMIJ)
    
    def determineprice(self):
        for keys,values in self.winningSeller.items():
            winnername = list(self.winningBuyer[keys].keys())[0]
            temp = 0
            if len(values[0])>0:
                for i in values[0]:
                        self.csps[i].pays[1]=self.csps[i].pays[1] + self.alphaMIJ[keys][winnername][i]*values[1]
                        temp = temp + self.alphaMIJ[keys][winnername][i]
                        
            self.csps[winnername].pays[4] = self.csps[winnername].pays[4]+ temp * self.winningBuyer[keys][winnername]
                    
            
    

In [9]:
def determineMu(csps,auc):
    possibleService=csps[0].obtainServiceSdesc()
    
    muSIT = [[[0 for i in csps] for j in csps] for k in range(csps[0].typeofServiceS)]

    for key,value in auc.alphaMIJ.items():
        winningBuy=list(auc.winningBuyer[key].keys())
        winningSell=list(auc.winningSeller[key][0])
        servtype = int(csps[winningBuy[0]].sStarM[key])
        gs = possibleService[servtype][1]
        msInd= csps[0].possibleVMtype.index(key)
        value = [[x//gs for x in y] for y in value] 
        for i in csps:
            if i.name in winningBuy or i.name not in winningSell:
                if ((i.quelen[servtype]+i.vQue[servtype])/gs) > (i.V*i.betaIT/i.cMI[msInd]) :
                    value[i.name][i.name]=i.notosell(msInd)//gs
        muSIT[servtype]=value

    return muSIT

In [10]:
def noOfactivatedServers(muSIT,csps,auc):
    possibleService=csps[0].obtainServiceSdesc()
    activeNMI={}
    sumAlpha={}

    for key,value in auc.alphaMIJ.items():
        temp=[0 for i in csps]
        sumAlpha[key]=[]
        for cloud in value:
            temp = [(temp[i]+cloud[i]) for i in range(len(csps))]
        sumAlpha[key]=temp.copy()    
        
    sumMuSII  = {} 
    mS = -1
    for servtype in range(len(muSIT)):
        if mS != possibleService[servtype][0]:
            mS=possibleService[servtype][0]
            sumMuSII[mS]=[]
            temp=[0 for i in csps]

        for j in range(len(muSIT[servtype])):
            temp[j] = temp[j]+muSIT[servtype][j][j]*possibleService[servtype][1]
            
        sumMuSII[mS]= temp.copy()
    
    for key,value in sumMuSII.items():
        activeNMI[key]=[(value[i]+sumAlpha[key][i])//csps[0].cMI[csps[0].possibleVMtype.index(key)] for i in range(len(csps)) ]
            
    return activeNMI       

In [11]:
def auctionprocess(csps):
    auc = auctioneer(csps)
    auc.allbidsandasks()
    auc.sortBid()
#     auc.printbids()
    auc.winnerDetermination()
    auc.determineAlpha()
    auc.determineprice()
    return auc

In [12]:
if __name__ =="__main__":
    print("    ROUND-0   \n")
    noofClouds = 10
    possibleVMtypes = ['ss','sm','sl','ls','lm','ll']
    possibleDS = [150,300]
    possiblenoofVM = [1,2]
    VMprice = [58.75,97,173.51,76.26,117.51,194] #values are in $/months(730 hours)
    VMpricepH = [ i/730 for i in VMprice]
    clouds = [i for i in range(noofClouds)]
    
    CSP = []

    for i in range(10):
        CSP.append(CSPs(i,np.random.randint(800,1000, size=6).tolist(),random.uniform(0.3,0.4),VMpricepH))
    

    timeslot = -1
    request = jobgenerator(data,timeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM)
    
    
    
#resource request per round for s type of job
    rsit = [[] for i in CSP]

    for req in request: 
        rsit[req.cloud].append(req)   
    
    for c in CSP:
        c.initqueue(rsit)
        c.getBids()
        
    auc= auctionprocess(CSP)
    
    muSIT = determineMu(CSP,auc)
#     print("\n# of jobs i->j:",muSIT)
    active=noOfactivatedServers(muSIT,CSP,auc)  
#     print("\n# of Active Servers:",active)
    for i in CSP:
        i.queueupdate(muSIT,rsit)
   


    timeperiod = 2000
    for t in range(timeperiod):
        print(f"\n      ROUND-{t+1}    \n ")
        newtimeslot = t%696
        request = jobgenerator(data,newtimeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM)

        rsit = [[] for i in CSP]

        for req in request: 
            rsit[req.cloud].append(req)   
    
        for c in CSP:
            c.getBids()

        auc= auctionprocess(CSP)
        
        muSIT = determineMu(CSP,auc)
#         print("\n# of jobs i->j:",muSIT)
        active=noOfactivatedServers(muSIT,CSP,auc)  
#         print("\n# of Active Servers:",active)
#updating the queue
        for i in CSP:
            i.dropjobs()
            i.vqueueupdate(muSIT,CSP)
            i.queueupdate(muSIT,rsit)
            i.provisioningcost(active)
            
    
#     temp = {} 
#     temp1={}
#     for i in CSP:
# #         print("total no. of job drop: ",i.pays)
        
# #         i.dropinend()
# #         print("quuelen :",i.quelen)
# #         print("total no. of job drop: ",i.pays)
#         i.totalpayment()
#         temp[f"CSP{i.name}"] = i.totalpay//timeperiod
#         temp1[f"CSP{i.name}"] = i.D
# #         print(f"total pay of {i.name} : ",i.totalpay//timeperiod)
# #         print(f"total pay of {i.name} : ",i.totalpay)
        

#     print(temp) 
    
    

    ROUND-0   

WinningBuyer: {'ss': {6: 0.0001287777777777778}, 'sm': {5: 5.688888888888889e-05}, 'sl': {7: 2.7e-05}, 'ls': {1: 1.5333333333333334e-05}, 'lm': {6: 0.00012377777777777777}, 'll': {4: 0.00019088888888888888}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-1    
 
WinningBuyer: {'ss': {6: 0.0002575555555555556}, 'sm': {5: 0.00011377777777777778}, 'sl': {7: 5.4e-05}, 'ls': {1: 3.066666666666667e-05}, 'lm': {6: 0.00024755555555555555}, 'll': {4: 0.00038177777777777776}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-2    
 
WinningBuyer: {'ss': {6: 0.00037811111111111113}, 'sm': {4: 0.00023655555555555555}, 'sl': {7: 0.00017011111111111111}, 'ls': {4: 0.00014022222222222222}, 'lm': {6: 0.0003858888888888889}, 'll': {4: 0.0004952222222222222}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 


      ROUND-25    
 
WinningBuyer: {'ss': {6: 0.0038046666666666667}, 'sm': {9: 0.0031386666666666668}, 'sl': {7: 0.0029464444444444442}, 'ls': {1: 0.002954888888888889}, 'lm': {6: 0.0034605555555555557}, 'll': {4: 0.003914444444444444}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-26    
 
WinningBuyer: {'ss': {6: 0.0040698888888888885}, 'sm': {9: 0.0032655555555555554}, 'sl': {9: 0.0030718888888888888}, 'ls': {1: 0.0030695555555555554}, 'lm': {6: 0.0036474444444444445}, 'll': {4: 0.0040847777777777775}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-27    
 
WinningBuyer: {'ss': {6: 0.004991777777777778}, 'sm': {9: 0.0034011111111111113}, 'sl': {7: 0.0031953333333333334}, 'ls': {1: 0.0031884444444444443}, 'lm': {6: 0.0037923333333333333}, 'll': {5: 0.004321}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm':


      ROUND-49    
 
WinningBuyer: {'ss': {7: 0.009859111111111112}, 'sm': {9: 0.006243777777777778}, 'sl': {7: 0.005897}, 'ls': {1: 0.005839777777777778}, 'lm': {8: 0.006945333333333334}, 'll': {5: 0.009824111111111112}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-50    
 
WinningBuyer: {'ss': {7: 0.010014}, 'sm': {9: 0.006377666666666667}, 'sl': {7: 0.006018777777777777}, 'ls': {7: 0.005992555555555556}, 'lm': {8: 0.007132111111111111}, 'll': {2: 0.010196}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-51    
 
WinningBuyer: {'ss': {7: 0.010154555555555556}, 'sm': {9: 0.006509}, 'sl': {7: 0.006145}, 'ls': {7: 0.006115777777777778}, 'lm': {8: 0.0072848888888888885}, 'll': {2: 0.010562222222222223}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-52    
 
WinningBuyer: 

WinningBuyer: {'ss': {1: 0.010093111111111112}, 'sm': {9: 0.009607888888888888}, 'sl': {7: 0.008946666666666667}, 'ls': {9: 0.008919666666666666}, 'lm': {8: 0.010699888888888889}, 'll': {8: 0.015848333333333332}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-75    
 
WinningBuyer: {'ss': {1: 0.010164111111111112}, 'sm': {9: 0.009734444444444445}, 'sl': {7: 0.009067666666666667}, 'ls': {9: 0.009046222222222223}, 'lm': {8: 0.010831555555555556}, 'll': {8: 0.016023555555555555}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-76    
 
WinningBuyer: {'ss': {1: 0.010282777777777778}, 'sm': {9: 0.009864111111111112}, 'sl': {7: 0.009182}, 'ls': {9: 0.009175}, 'lm': {8: 0.010951777777777777}, 'll': {8: 0.016180333333333335}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-77    
 
W

WinningBuyer: {'ss': {8: 0.011042277777777778}, 'sm': {3: 0.008015333333333333}, 'sl': {4: 0.008123777777777778}, 'ls': {9: 0.011925444444444445}, 'lm': {9: 0.014093555555555555}, 'll': {8: 0.019224}}
WinningSeller: {'ss': [[], 0.010890904771356054], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-99    
 
WinningBuyer: {'ss': {8: 0.011107333333333334}, 'sm': {3: 0.008154222222222222}, 'sl': {4: 0.008239}, 'ls': {9: 0.012069}, 'lm': {9: 0.014238666666666667}, 'll': {8: 0.01934311111111111}}
WinningSeller: {'ss': [[0, 5], 0.011107166666666666], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-100    
 
WinningBuyer: {'ss': {6: 0.011188166666666667}, 'sm': {3: 0.008279888888888889}, 'sl': {4: 0.008352444444444444}, 'ls': {9: 0.012207666666666667}, 'lm': {9: 0.014369888888888889}, 'll': {8: 0.019457222222222223}}
WinningSeller: {'ss': [[0, 2], 0.011159333333333334], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 

...

WinningBuyer: {'ss': {2: 0.011130444444444445}, 'sm': {0: 0.010950111111111112}, 'sl': {5: 0.011332611111111111}, 'ls': {1: 0.03634805555555556}, 'lm': {9: 0.03618811111111111}, 'll': {7: 0.03629011111111111}}
WinningSeller: {'ss': [[5], 0.011130444444444445], 'sm': [[], 0], 'sl': [[0], 0.011332611111111111], 'ls': [[0, 2, 6, 3], 0.03623722222222222], 'lm': [[5, 0, 6, 3], 0.036179444444444445], 'll': [[0, 5, 3, 2], 0.03629011111111111]}

      ROUND-1925    
 
WinningBuyer: {'ss': {0: 0.011146777777777778}, 'sm': {4: 0.010998333333333334}, 'sl': {2: 0.011264944444444444}, 'ls': {7: 0.03636938888888889}, 'lm': {8: 0.036242166666666666}, 'll': {6: 0.03628833333333333}}
WinningSeller: {'ss': [[5], 0.011072418241418214], 'sm': [[], 0.010890904771356054], 'sl': [[0], 0.011056409629501192], 'ls': [[0, 2, 5, 6], 0.03625827777777778], 'lm': [[5, 0, 6, 2], 0.036242166666666666], 'll': [[0, 5, 2], 0.03628833333333333]}

      ROUND-1926    
 
WinningBuyer: {'ss': {8: 0.011103}, 'sm': {4: 0.01106

WinningBuyer: {'ss': {1: 0.011326611111111112}, 'sm': {4: 0.01114111111111111}, 'sl': {5: 0.011316055555555555}, 'ls': {0: 0.03645544444444444}, 'lm': {9: 0.03624616666666667}, 'll': {4: 0.036317}}
WinningSeller: {'ss': [[0, 5], 0.011072418241418214], 'sm': [[0, 2], 0.01114111111111111], 'sl': [[2], 0.011216833333333334], 'ls': [[3, 2, 6, 4], 0.03645544444444444], 'lm': [[2, 5, 0, 3], 0.036061152113644176], 'll': [[0, 3, 6, 5], 0.036317]}

      ROUND-1943    
 
WinningBuyer: {'ss': {7: 0.0113615}, 'sm': {5: 0.011101555555555555}, 'sl': {9: 0.011306555555555556}, 'ls': {7: 0.03641144444444445}, 'lm': {8: 0.036222222222222225}, 'll': {4: 0.03636383333333333}}
WinningSeller: {'ss': [[0, 5], 0.011072418241418214], 'sm': [[0], 0.011072418241418214], 'sl': [[5, 2], 0.011253277777777777], 'ls': [[5, 3, 0, 2], 0.036388166666666666], 'lm': [[2, 5, 0, 6], 0.03606461111111111], 'll': [[0, 3, 6, 5], 0.03636383333333333]}

      ROUND-1944    
 
WinningBuyer: {'ss': {4: 0.011155333333333333}, 'sm'

WinningBuyer: {'ss': {5: 0.011099833333333333}, 'sm': {8: 0.011544444444444444}, 'sl': {7: 0.011161111111111111}, 'ls': {6: 0.03664755555555556}, 'lm': {8: 0.03652155555555556}, 'll': {9: 0.03661855555555556}}
WinningSeller: {'ss': [[0], 0.011099833333333333], 'sm': [[0, 5], 0.011072418241418214], 'sl': [[0, 5], 0.011072418241418214], 'ls': [[5, 0, 2, 4], 0.036555166666666666], 'lm': [[2, 0, 5, 3, 4], 0.03652155555555556], 'll': [[2, 5, 3, 6, 0], 0.03661855555555556]}

      ROUND-1961    
 
WinningBuyer: {'ss': {3: 0.010628944444444444}, 'sm': {8: 0.0115795}, 'sl': {9: 0.01117061111111111}, 'ls': {6: 0.03669794444444444}, 'lm': {6: 0.036406666666666664}, 'll': {0: 0.03666277777777778}}
WinningSeller: {'ss': [[], 0], 'sm': [[0, 5], 0.011111277777777777], 'sl': [[0, 5], 0.011072418241418214], 'ls': [[5, 0, 2, 4], 0.03663311111111111], 'lm': [[2, 0, 5], 0.036406666666666664], 'll': [[2, 5, 3, 6], 0.03643241103306005]}

      ROUND-1962    
 
WinningBuyer: {'ss': {3: 0.01069488888888889},

WinningBuyer: {'ss': {4: 0.011127277777777778}, 'sm': {9: 0.011082333333333333}, 'sl': {9: 0.010914722222222221}, 'ls': {1: 0.03671666666666667}, 'lm': {0: 0.036451833333333336}, 'll': {4: 0.036569222222222225}}
WinningSeller: {'ss': [[0, 2], 0.011103777777777778], 'sm': [[0, 5], 0.011082333333333333], 'sl': [[], 0.010890904771356054], 'ls': [[3, 6, 4, 0, 5], 0.03671666666666667], 'lm': [[5, 3, 4, 2], 0.036451833333333336], 'll': [[5, 3, 0, 6], 0.036569222222222225]}

      ROUND-1979    
 
WinningBuyer: {'ss': {4: 0.011197166666666666}, 'sm': {5: 0.011131}, 'sl': {9: 0.010946166666666667}, 'ls': {2: 0.036716}, 'lm': {2: 0.03648005555555556}, 'll': {2: 0.03635527777777778}}
WinningSeller: {'ss': [[0, 2], 0.011187333333333334], 'sm': [[0], 0.011131], 'sl': [[], 0.010918111111111111], 'ls': [[3, 6, 4, 0], 0.036716], 'lm': [[0, 5, 3, 6], 0.03643241103306005], 'll': [[5, 3, 0], 0.03623138888888889]}

      ROUND-1980    
 
WinningBuyer: {'ss': {1: 0.011145}, 'sm': {7: 0.011175555555555555}

WinningBuyer: {'ss': {7: 0.011120277777777778}, 'sm': {4: 0.011017777777777777}, 'sl': {4: 0.011079555555555556}, 'ls': {8: 0.036423333333333335}, 'lm': {0: 0.03598205555555556}, 'll': {8: 0.036421166666666664}}
WinningSeller: {'ss': [[0, 5], 0.011072418241418214], 'sm': [[], 0.010890904771356054], 'sl': [[0, 5], 0.011079555555555556], 'ls': [[5, 0, 2, 6], 0.03616911111111111], 'lm': [[5], 0.035770166666666665], 'll': [[0, 5, 6, 3], 0.036421166666666664]}

      ROUND-1997    
 
WinningBuyer: {'ss': {6: 0.011122888888888889}, 'sm': {4: 0.011170222222222222}, 'sl': {2: 0.011074166666666666}, 'ls': {9: 0.03639977777777778}, 'lm': {1: 0.03590422222222222}, 'll': {2: 0.03641311111111111}}
WinningSeller: {'ss': [[5, 0], 0.011098611111111111], 'sm': [[0, 5], 0.011072418241418214], 'sl': [[0], 0.011074166666666666], 'ls': [[5, 0, 6, 2], 0.03623066666666667], 'lm': [[5, 2], 0.03567122222222222], 'll': [[5, 0, 6], 0.03613522222222222]}

      ROUND-1998    
 
WinningBuyer: {'ss': {6: 0.01120977

In [13]:
temp1={}
temp = {}
temp2 ={}
for i in CSP:
    i.totalpayment()
    temp[f"CSP{i.name}"] = i.totalpay//timeperiod
    temp1[f"CSP{i.name}"] = i.D
    temp2[f'CSP{i.name}'] = i.pays

In [14]:
item = pd.DataFrame.from_dict(temp, orient='index')
item1 = pd.DataFrame.from_dict(temp1, orient='index')
item2 = pd.DataFrame.from_dict(temp2, orient='index')

writer = pd.ExcelWriter(f'./outputpay/paydata9_2000.xlsx', engine='xlsxwriter')
item.to_excel(writer,sheet_name="pay of all clouds")
item1.to_excel(writer,sheet_name="no of job drop")
item2.to_excel(writer,sheet_name="All Pays")
writer.save()

In [15]:
for i in CSP:
    print(f"paysOff of ${i.name}:",i.pays)

paysOff of $0: [3090587.138685834, 2120718.4332767124, 2522203.8343703286, 0, 381053.7722327778]
paysOff of $1: [3089072.9991243496, 77434.47014333337, 80589.02697636254, 0, 956905.3876666672]
paysOff of $2: [3085468.999370539, 1564590.7482383018, 1990785.1466201635, 0, 404069.53144944424]
paysOff of $3: [3085455.6083158585, 775231.2041293802, 965064.6711703957, 0, 770673.0101288887]
paysOff of $4: [3086081.967562587, 212422.57665779078, 250644.4225082507, 0, 924396.3148966671]
paysOff of $5: [3083742.4507809673, 1954388.5551211115, 2371619.103680881, 0, 416081.2776038893]
paysOff of $6: [3088843.9933298263, 619964.2138565108, 769878.5517621927, 0, 799185.9980688895]
paysOff of $7: [3083427.4293707483, 62559.76004000001, 64788.08179308058, 0, 957277.3537944449]
paysOff of $8: [3088152.460617846, 69832.46256638262, 71861.56307038087, 0, 968814.2489794451]
paysOff of $9: [3087835.8827815633, 57431.108053333366, 57633.55369912515, 0, 970815.5125672226]


In [16]:
# for i in CSP:
#     print(i.D)



In [17]:
# condition = data["timeslot"] < 101 
# req = data[condition]
# # print(df)
# condition =  req['sizecpu']=='s'
# req = req[condition].copy()
# condition =  req['sizeram']=='l'
# req = req[condition].copy()
# print(len(req))
